# Customizing shooting moves: Analysis

Now that you've run the simulation, let's compare the results of two-way shooting with uniform shooting point selection to those with biased shooting point selection.

We'll open the output file and consider a few results. Although you can perform all the typical analyses you might do with TPS data (path densities, etc.), here we're just going to look at the acceptance probabilities and the shooting point distributions.

In [ ]:
import matplotlib.pyplot as plt

import openpathsampling as paths

In [ ]:
std_two_way = paths.Storage("2_way.nc", mode='r')
biased_two_way = paths.Storage("biased.nc", mode='r')

In [ ]:
two_way_scheme = std_two_way.schemes['2_way']
biased_scheme = biased_two_way.schemes['biased_shooting']

In [ ]:
two_way_scheme.move_summary(std_two_way.steps)

In [ ]:
biased_scheme.move_summary(biased_two_way.steps)

As you can see, the acceptance rate for uniform 2-way shooting with complete velocity randomization is pretty low. We improve this significantly by using the Gaussian biased shooting (keeping in mind that 25% is the theoretical maximum acceptance rate for the approach we use here.)

Next we will plot the shooting points with uniform shooting point selection and with the Gaussian biased shooting point selection. First we write and use a little function to extract the shooting points.

In [ ]:
def get_shooting_points(steps):
    """Function to extract x,y positions of all shooting points"""
    shooting_snaps = [step.change.canonical.details.shooting_snapshot for step in steps]
    xy = [snap.xyz[0][:2] for snap in shooting_snaps]  # get x and y positions
    return tuple(zip(*xy))  # [[x1, y1], [x2, y2]]  => ([x1, x2], [y1, y2])

In [ ]:
# the 0th step saves initial conditions, so we only have shooting moves as of step 1
std_x, std_y = get_shooting_points(std_two_way.steps[1:])
biased_x, biased_y = get_shooting_points(biased_two_way.steps[1:])

The next two plots show the location of shooting points in the when using uniform shooting (first plot) and when using Gaussian biased shooting (second plot). Note that the shooting points fall in a much narrower range in the Gaussian biased plots. This is also why we get a better acceptance rate!

In [ ]:
%run toy_plot_helpers.py
# doesn't matter where we get the toy engine from; same in both
engine = std_two_way.engines['toy_engine']
pes = engine.topology.pes
plot = ToyPlot()
plot.contour_range = np.arange(-1.5, 1.0, 0.1)
plot.add_pes(pes)

In [ ]:
# using c=range(len(std_x) and cmap='rainbow' here means that color corresponds to step order
plot.plot()
plt.scatter(std_x, std_y, c=range(len(std_x)), cmap='rainbow');

In [ ]:
plot.plot()
plt.scatter(biased_x, biased_y, c=range(len(std_x)), cmap='rainbow');

* How would you expect the shooting acceptance to change if we changed $l_0$, the center of the Gaussian? What useful information do we have about this potential energy surface that we might not have in a real system?
* What would be the effect of changing the Gaussian selector's $\alpha$ value (Gaussian width parameter)?

Now let's make similar plots, but we'll plot the shooting points for accepted/rejected steps in different colors.

In [ ]:
# create lists to plot the accepted shooting points
std_acc_steps = [step for step in std_two_way.steps[1:] if step.change.accepted]
std_acc_x, std_acc_y = get_shooting_points(std_acc_steps)

biased_acc_steps = [step for step in biased_two_way.steps[1:] if step.change.accepted]
biased_acc_x, biased_acc_y = get_shooting_points(biased_acc_steps)

In [ ]:
# YOUR TURN: Do the same for the rejected shooting points.

# std_rej_x, std_rej_y = ...
# biased_rej_x, biased_rej_y = ...

In [ ]:
plot.plot()
plt.scatter(std_rej_x, std_rej_y, c='r', label='Rejected')
plt.scatter(std_acc_x, std_acc_y, c='b', label='Accepted')
plt.legend();

In [ ]:
plot.plot()
plt.scatter(biased_rej_x, biased_rej_y, c='r', label='Rejected')
plt.scatter(biased_acc_x, biased_acc_y, c='b', label='Accepted')
plt.legend();

* Are the accepted are rejected shooting points distributed similarly with uniform shooting? What about with the biased selector?
* Given the plot of accepted and rejected shooting points for uniform shooting, what would you propose as a shooting point selection bias?